In [ ]:
import numpy as np
import skvideo.io
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.measure import label
from skimage.morphology import remove_small_objects, erosion, dilation, disk

from trackbox import *

### 0. Load the video
* Change `filename = "name_of_video.wmv"` to the name of your own video.
* Change `subsample=1` to `n` to speed-up the video by `n` times.
* Change `start` and `end` to the MM:SS format (e.g., `3:11`) to trim the video.

In [ ]:
filename = "name_of_video.wmv"
height, width = 28, 65 # height and width of the chamber in cm
kwargs = {
    "subsample": 2,   # subsampling ratio of the video
    "start": "0:09",  # start time of the trimmed video 
    "end": "4:09",    # end time of the trimmed video 
}
video, video_gray, frame_rate = load_video(filename, **kwargs)
show_rand_imgs(video, 3)
show_rand_imgs(video_gray, 3, 'gray')

### 1. Find valid region in the field-of-view (FoV)

In [ ]:
background = np.median(video_gray, axis=0)
valid_region, pix_height, pix_width = find_valid_region(background)
plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(background, cmap='gray')
plt.axis('off')
plt.title('Background')
plt.subplot(122)
plt.imshow(valid_region, cmap='gray')
plt.axis('off')
plt.title('Valid region')
plt.show()
perpix_dist = ((height / pix_height) + 
               (width / pix_width)) / 2.0

### 2. Process and save the video

In [ ]:
all_centers = []
center_video = np.zeros(video_gray.shape, dtype=np.uint8)
for i in list(range(video_gray.shape[0])):
    center = segment_image(video_gray[i]-background, show_imgs=False, valid_region=valid_region)
    if len(center)==2:
        center_video[i, center[0], center[1]] = 1
        all_centers.append(center)
save_video(video, video_gray, center_video, track=True)

In [ ]:
travel_length = 0
temp = np.ones(video_gray[0].shape, dtype=np.uint8)*255
temp[0,0] = 0
plt.imshow(temp, cmap='gray')
for j in range(len(all_centers)-1):
    dist = center2dist(all_centers[j], all_centers[j+1])
    if dist < 100:
        travel_length += dist
        plt.plot([all_centers[j][1], 
                  all_centers[j+1][1]], 
                 [all_centers[j][0],
                  all_centers[j+1][0]], 'r', alpha=0.3, linewidth=2)
plt.axis('on')
plt.show()
print("Travel length is %.2f pixels, which is %.2f cm." % (travel_length, travel_length*perpix_dist))

In [ ]:
## This cell is intentionally left blank.